In [2]:
import os
import pickle
import itertools
from pytransform3d.transform_manager import TransformManager
import numpy as np
import json

In [3]:
root = "/nas/EOS/servers/storage03-srv2/dataset/axcar/stereo"
capture = 'CAR-359'

In [4]:
data_dict = {}
data_dict[capture] = {}

In [5]:
def load_pickle(filepath):
    with open(filepath, 'rb') as fh:
        data = pickle.load(fh)
    return data

In [6]:
def decompose_affine_mat(T):
    return T[0:3,0:3], T[0:3,3]

In [7]:
# scan a capture for type of cameras
os.listdir(os.path.join(root,capture, 'calibration_matrix'))

['camera_extrinsic.json',
 'lidar_extrinsic.json',
 'lidar_extrinsic.pickle',
 'camera_intrinsic.pickle',
 'camera_intrinsic.json',
 'camera_extrinsic.pickle']

In [8]:
intr_data = load_pickle(os.path.join(root,capture, 'calibration_matrix', 'camera_intrinsic.pickle'))

In [9]:
extr_data = load_pickle(os.path.join(root,capture, 'calibration_matrix', 'camera_extrinsic.pickle'))

In [10]:
extr_data

{'ar0231_0_ar0231_1': {'rvec': array([[ 0.99890783, -0.02253468,  0.04093098],
         [ 0.02427365,  0.99880166, -0.04249741],
         [-0.03992426,  0.04344454,  0.99825779]]),
  'tvec': array([[-233.9825111 ],
         [ -14.36727582],
         [   3.485073  ]])},
 'ar0231_0_ar0231_2': {'rvec': array([[ 0.99782838, -0.00811658,  0.06536546],
         [ 0.00948823,  0.99974068, -0.02070128],
         [-0.06518048,  0.02127652,  0.99764664]]),
  'tvec': array([[ 400.16291121],
         [-227.21004322],
         [-485.79114177]])},
 'ar0231_2_ar0231_3': {'rvec': array([[ 0.99933144, -0.01851419, -0.03152603],
         [ 0.01869952,  0.9998095 ,  0.00559408],
         [ 0.03141645, -0.00617986,  0.99948728]]),
  'tvec': array([[-1063.09456259],
         [  -34.74242633],
         [  -24.5380133 ]])},
 'ar0231_0_ar0231_3': {'rvec': array([[ 0.99903185, -0.02798316,  0.03394553],
         [ 0.02845867,  0.99950236, -0.01360667],
         [-0.03354788,  0.01455954,  0.99933106]]),
  'tve

In [11]:
cams = list(extr_data.keys()) 

In [31]:
cams

['ar0231_0_ar0231_1',
 'ar0231_0_ar0231_2',
 'ar0231_2_ar0231_3',
 'ar0231_0_ar0231_3',
 'ar0231_0_fsc231_0',
 'ar0231_0_fsc231_1',
 'fsc231_0_fsc231_1']

In [12]:
tm = TransformManager()
for cam_pair in list(extr_data.keys()): 
    cam1, cam2 = tuple([f'{cam}_{num}' for cam,num in  zip(cam_pair.split('_')[::2],cam_pair.split('_')[1::2])])
    # print(cam1, cam2)

    # extract transformation    
    T = np.eye(4)
    R, t = extr_data[cam_pair]['rvec'].reshape(3,3), extr_data[cam_pair]['tvec'].reshape(3,)
    T[0:3,0:3] = R

    if np.linalg.norm(t) > 10:
        T[0:3,3] = t/1000
    else:
        T[0:3,3] = t
    
    # Add transformation to pytransform3d
    tm.add_transform(cam1, cam2, T)

In [13]:
print(tm.get_transform('ar0231_2','ar0231_3'))

[[ 0.99933144 -0.01851419 -0.03152603 -1.06309456]
 [ 0.01869952  0.9998095   0.00559408 -0.03474243]
 [ 0.03141645 -0.00617986  0.99948728 -0.02453801]
 [ 0.          0.          0.          1.        ]]


In [14]:
T = np.eye(4)
R, t = extr_data[cam_pair]['rvec'].reshape(3,3), extr_data[cam_pair]['tvec'].reshape(3,)
T[0:3,0:3] = R

if np.linalg.norm(t) > 10:
    T[0:3,3] = t/1000
else:
    T[0:3,3] = t
    
print(T)

[[ 0.99984009 -0.00250018 -0.01770736 -0.84537773]
 [ 0.0028989   0.99974202  0.02252759  0.00665   ]
 [ 0.01764647 -0.02257532  0.99958939 -0.03833026]
 [ 0.          0.          0.          1.        ]]


In [15]:
np.linalg.norm(t) > 10

True

In [16]:
cam_types = set(map(lambda x : x.split('_')[0], intr_data.keys()))

for cam_type in cam_types:
    data_dict[capture][cam_type] = { 'intrinsics': {}, 'extrinsics': {} } 

In [32]:
cam_types

{'ar0231', 'fsc231'}

In [17]:
intr_data

{'ar0231_0': {'cam_mat': array([[952.2730727 ,   0.        , 973.47716632],
         [  0.        , 952.18703765, 621.44828005],
         [  0.        ,   0.        ,   1.        ]]),
  'dist': array([[ 2.1962741e-01, -1.0277259e-02,  0.0000000e+00,  0.0000000e+00,
           3.5599177e-04,  5.7432318e-01,  0.0000000e+00,  0.0000000e+00]],
        dtype=float32)},
 'ar0231_1': {'cam_mat': array([[955.92076911,   0.        , 977.18382566],
         [  0.        , 955.34951347, 604.93003406],
         [  0.        ,   0.        ,   1.        ]]),
  'dist': array([[ 2.4873178e-01, -1.4032701e-02,  0.0000000e+00,  0.0000000e+00,
           5.6917069e-04,  6.0906255e-01,  0.0000000e+00,  0.0000000e+00]],
        dtype=float32)},
 'ar0231_2': {'cam_mat': array([[1.95306529e+03, 0.00000000e+00, 9.74729343e+02],
         [0.00000000e+00, 1.95157780e+03, 6.05948999e+02],
         [0.00000000e+00, 0.00000000e+00, 1.00000000e+00]]),
  'dist': array([[-0.548534  ,  0.24690242,  0.        ,  0.    

In [18]:
intr_data['fsc231_1']['dist'].shape

(1, 14)

In [19]:
cam_data = {cam_type : [tag.split('_')[1] for tag in intr_data.keys() if cam_type in tag] for cam_type in cam_types}

In [20]:
data_dict

{'CAR-359': {'ar0231': {'intrinsics': {}, 'extrinsics': {}},
  'fsc231': {'intrinsics': {}, 'extrinsics': {}}}}

In [21]:
for cam_type in cam_types:
    cam_nums = cam_data[cam_type]
    for cam_num in cam_nums:
        # intrinsics data
        data_dict[capture][cam_type]['intrinsics'][f'{cam_num}'] = {'K'    : list(intr_data[f'{cam_type}_{cam_num}']['cam_mat'].astype(float).reshape(-1)), 
                                                                    'dist' : list(intr_data[f'{cam_type}_{cam_num}']['dist'].astype(float).reshape(-1))}
    # extriniscs data
    cam_pairs  = [f'{a}_{b}' for a,b in itertools.combinations(cam_data[cam_type], r = 2)]
    # print(cam_pairs)
    for cam_pair in cam_pairs:
        R, tvec = decompose_affine_mat(tm.get_transform(f"{cam_type}_{cam_pair.split('_')[0]}",
                                                        f"{cam_type}_{cam_pair.split('_')[1]}"))
        
        data_dict[capture][cam_type]['extrinsics'][cam_pair] = {
                                                                'rmat' : list(R.reshape(-1)), 
                                                                'tvec' : list(tvec.reshape(-1))
                                                               }

In [22]:
f'{cam_type}__{cam_pair}'

'fsc231__0_1'

In [23]:
data_dict[capture][cam_type]

{'intrinsics': {'0': {'K': [7203.82031782057,
    0.0,
    1835.5012484945312,
    0.0,
    7209.87350148436,
    1069.2588144105162,
    0.0,
    0.0,
    1.0],
   'dist': [-0.20436932665353066,
    0.3524386662290267,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0]},
  '1': {'K': [7136.3619475835485,
    0.0,
    1816.0883556332105,
    0.0,
    7134.168540246251,
    837.1057492474125,
    0.0,
    0.0,
    1.0],
   'dist': [-0.20793085661457059,
    0.610783608258124,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0]}},
 'extrinsics': {'0_1': {'rmat': [0.9998400864039769,
    -0.0025001788596346504,
    -0.017707363591395155,
    0.0028989016671859876,
    0.9997420187440642,
    0.022527590343112837,
    0.017646472418362178,
    -0.022575319780967898,
    0.9995893941754159],
   'tvec': [-0.845377728245108, 0.006649997740926828, -0.03833025954325985]}}}

In [24]:
data_dict

{'CAR-359': {'ar0231': {'intrinsics': {'0': {'K': [952.2730726983133,
      0.0,
      973.4771663244936,
      0.0,
      952.1870376542149,
      621.4482800496887,
      0.0,
      0.0,
      1.0],
     'dist': [0.21962741017341614,
      -0.010277259163558483,
      0.0,
      0.0,
      0.000355991767719388,
      0.5743231773376465,
      0.0,
      0.0]},
    '1': {'K': [955.9207691107338,
      0.0,
      977.1838256620462,
      0.0,
      955.3495134688142,
      604.9300340582665,
      0.0,
      0.0,
      1.0],
     'dist': [0.24873177707195282,
      -0.014032701030373573,
      0.0,
      0.0,
      0.0005691706901416183,
      0.6090625524520874,
      0.0,
      0.0]},
    '2': {'K': [1953.065286975297,
      0.0,
      974.729343294804,
      0.0,
      1951.5777999488055,
      605.9489989408094,
      0.0,
      0.0,
      1.0],
     'dist': [-0.5485339760780334,
      0.24690242111682892,
      0.0,
      0.0,
      -0.007938316091895103,
      0.00277535733766853

In [30]:
with open(f'{capture}.json','w') as fh:
    json.dump(data_dict, fh, sort_keys=True, indent=4)

In [26]:
cam_data = {cam_type : [tag.split('_')[1] for tag in intr_data.keys() if cam_type in tag] for cam_type in cam_types}

In [27]:
intr_data.keys()

dict_keys(['ar0231_0', 'ar0231_1', 'ar0231_2', 'ar0231_3', 'fsc231_0', 'fsc231_1'])

In [28]:
cam_types

{'ar0231', 'fsc231'}

In [29]:
cam_data

{'ar0231': ['0', '1', '2', '3'], 'fsc231': ['0', '1']}

In [ ]:
def cartesian_prod(set_lists):
    if len(set_lists) == 1:
        return set_lists[0]
    else:
        left_set = set_lists[0]
        right_set = set_lists[1:]
        